This is my second Kaggle Competition I have learned a lot from the First which was "Titanic" I would suggest the beginners to go through Intro. and Intermediate ML course and apply those things in the Titanic Competition before doing this one.

In this Competition I try to make things more clear, precise and organise and perform better feature engineering than the Titanic Competition.

- Various libraries and use of scikit-learn

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Libraries and use of scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import itertools
from sklearn.metrics import mean_squared_error
import category_encoders 

# Loading data and extracting features
Here We will extract the columns or features who has missing values less than 800 and we will find out that intially we had 81 columns and after this operation we would left with 76 columns, 5 columns has missing values less than 800.

In [ ]:
train_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
print(train_data.info())
features=[cols for cols in train_data.columns if(train_data[cols].isnull().sum()<800)]
#we have choose 800 because fireplacequ has values 770 and we don't want to lose that information 

X=train_data[features]
y=X.SalePrice
X.drop(columns={'SalePrice'},inplace=True)
X_test=test_data[X.columns]
X.info()

## Feature Engineering
 We have 76 columns and now we will preprocess all of these,handling categorical and numerical data and fill the missing values to make it ready for Machine learning models.
 - Categorical data and missing values
 - numerical data and missing values

In [ ]:
test_id=X_test.Id
X.drop(columns={'Id'},inplace=True)
X_test.drop(columns={'Id'},inplace=True)
cat_col=[col for col in X.columns if X[col].dtype=='object']
num_col=[col for col in X.columns if X[col].dtype!='object']

In [ ]:
X_c=X.copy()
y_c=y.copy()
X_test_c=X_test.copy()
encoder=LabelEncoder()
for col in cat_col:
    X_c[col]=X_c[col].astype(str)
    X_test_c[col]=X_test_c[col].astype(str)
    X_c[col]=encoder.fit_transform(X_c[col])
    X_test_c[col]=encoder.fit_transform(X_test_c[col])

In [ ]:
# Replace the missing value with mode of that column and encoding by Label encoder
X_c1=X.copy()
y_c1=y.copy()
X_test_c1=X_test.copy()
missing_catcol=[col for col in X_c1[cat_col].columns if X_c1[col].isnull().any()==True]
X_c1[missing_catcol]=X_c1[missing_catcol].fillna(X_c1[missing_catcol].mode().iloc[0])
missing_catcol=[col for col in X_test_c1[cat_col].columns if X_test_c1[col].isnull().any()==True]
X_test_c1[missing_catcol]=X_test_c1[missing_catcol].fillna(X_test_c1[missing_catcol].mode().iloc[0])
for col in cat_col:
    X_c1[col]=encoder.fit_transform(X_c1[col])
    X_test_c1[col]=encoder.fit_transform(X_test_c1[col])

In [ ]:
# Replace the missing value with mode of that column and encoding by Count Encoder
X_c2=X.copy()
y_c2=y.copy()
X_test_c2=X_test.copy()
missing_catcol=[col for col in X_c2[cat_col].columns if X_c2[col].isnull().any()==True]
X_c2[missing_catcol]=X_c2[missing_catcol].fillna(X_c2[missing_catcol].mode().iloc[0])
missing_catcol=[col for col in X_test_c1[cat_col].columns if X_test_c1[col].isnull().any()==True]
X_test_c1[missing_catcol]=X_test_c1[missing_catcol].fillna(X_test_c1[missing_catcol].mode().iloc[0])
encoder2=category_encoders.CountEncoder()
X_c2[cat_col]=encoder2.fit_transform(X_c2[cat_col])
X_test_c2[cat_col]=encoder2.transform(X_test_c2[cat_col])


In [ ]:
#numerical data
imputer=SimpleImputer()
X_c[num_col]=pd.DataFrame(imputer.fit_transform(X_c1[num_col]))
X_test_c1[num_col]=pd.DataFrame(imputer.transform(X_test_c1[num_col]))

In [ ]:
#numerical data
X_c1[num_col]=pd.DataFrame(imputer.fit_transform(X_c[num_col]))
X_test_c[num_col]=pd.DataFrame(imputer.transform(X_test_c[num_col]))

In [ ]:
#numerical data
X_c2[num_col]=pd.DataFrame(imputer.fit_transform(X_c2[num_col]))
X_test_c2[num_col]=pd.DataFrame(imputer.transform(X_test_c2[num_col]))

# Model Selection
- Random Forest
- XGBoost

In [ ]:
#Random Forest on Label encoder and Simple Imputer
X_train,X_valid,y_train,y_valid=train_test_split(X_c,y_c)
model=RandomForestRegressor()
model.fit(X_train,y_train)
#print prediction for both training and validation data to get the idea of overfitting.
print(mean_absolute_error(model.predict(X_valid),y_valid))
print(mean_absolute_error(model.predict(X_train),y_train))

In [ ]:
X_train1,X_valid1,y_train1,y_valid1=train_test_split(X_c1,y_c1)
X_train2,X_valid2,y_train2,y_valid2=train_test_split(X_c2,y_c2)

In [ ]:
#We select the optimal value of estimators and learning rate to get ultimate model.

n_est=list(range(100,1500,200))
rate=list(np.arange(0.01,0.11,0.02))
value=list(itertools.product(n_est,rate))

In [ ]:
absolute={}
absolute1={}
for i in value:
    model=XGBRegressor(n_estimators=i[0],learning_rate=i[1])
    model.fit(X_train1, y_train1, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid1, y_valid1)],
             verbose=False)
    absolute[i]=mean_absolute_error(model.predict(X_valid1),y_valid1)
    model.fit(X_train2, y_train2, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid2, y_valid2)],
             verbose=False)
    absolute1[i]=mean_absolute_error(model.predict(X_valid2),y_valid2)
ult=min(absolute,key=absolute.get)
ult1=min(absolute1,key=absolute1.get)

In [ ]:
print("For X_c1 - {} error is {} \nFor X_c2- {} error is {} ".format(ult,absolute[ult],ult1,absolute1[ult1]))

In [ ]:
ult_model=XGBRegressor(n_estimators=ult1[0],learning_rate=ult[1])
ult_model.fit(X_train2, y_train2, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid2, y_valid2)],
             verbose=False)
print(mean_absolute_error(ult_model.predict(X_valid2),y_valid2))
print(mean_absolute_error(ult_model.predict(X_train2),y_train2))


In [ ]:
ult_model1=XGBRegressor(n_estimators=ult[0],learning_rate=ult[1])
ult_model1.fit(X_train1, y_train1, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid1, y_valid1)],
             verbose=False)
print(mean_absolute_error(ult_model1.predict(X_valid1),y_valid1))
print(mean_absolute_error(ult_model1.predict(X_train1),y_train1))

In [ ]:

predictions=ult_model.predict(X_test_c2)
output=pd.DataFrame({'Id':test_id,'SalePrice':predictions})
output.to_csv('sample_submission.csv',index=False)